[Reference](https://python.plainenglish.io/convert-microphone-input-to-text-generate-a-response-from-gpt-3-and-save-audio-to-a-file-using-cdd61c465d28)

In [1]:
!pip install SpeechRecognition
!pip install openai
!pip install gTTS
!pip install pyaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.5-py3-none-any.whl size=67620 sha256=c2f6976256f18e5373ad269abc914326161239ebca6115dcc80281041550dae3
  Stored in directory: /root/.cache/pip/wheels/a7/47/99/8273a59fbd59c

In [2]:
import speech_recognition as sr
import openai
import os
from gtts import gTTS
import pyaudio
import wave

ModuleNotFoundError: ignored

In [3]:
openai.api_key = os.environ["OPENAI_API_KEY"]

r = sr.Recognizer()
with sr.Microphone() as source:
    audio = r.listen(source)

text = r.recognize_google(audio)
print(text)

def generate_response(prompt):
    completions = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )
    message = completions.choices[0].text
    return message

response = generate_response(text)
print(response)

tts = gTTS(response)
tts.save("response.mp3")

os.system("response.mp3")

audio = pyaudio.PyAudio()

# Set the microphone as the audio source
input_device_index = None
for i in range(audio.get_device_count()):
    device_info = audio.get_device_info_by_index(i)
    if device_info["name"].lower() == "microphone":
        input_device_index = device_info["index"]
        break

if input_device_index is None:
    raise ValueError("No microphone was found")

stream = audio.open(
    format=pyaudio.paInt16,
    channels=1,
    rate=44100,
    input=True,
    input_device_index=input_device_index,
)

wavefile = wave.open("recording.wav", "wb")
wavefile.setnchannels(1)
wavefile.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
wavefile.setframerate(44100)

# Read audio from the microphone and save it to the file
data = stream.read(1024)
while data:a
    wavefile.writeframes(data)
    data = stream.read(1024)

wavefile.close()
stream.stop_stream()
stream.close()
audio.terminate()